In [0]:
from google.colab import files
from google.colab import drive
import os
import pandas as pd
import pickle
import os
import time
from statistics import mean
import math
from PIL import Image
import numpy as np
import pandas as pd
from IPython.display import display
import pickle as pkl
import multiprocessing
import threading
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [0]:
drive.mount('/content/drive/')
os.chdir('drive/My Drive/Winter 2020/MCA Assignments/Assignment-1/')

In [0]:
img = Image.open('images/all_souls_000000.jpg')
pixels = np.asarray(img.getdata())

In [0]:
def process(img_src):
    img = cv2.imread(img_src, 0)
    w, h = img.shape
    return img

In [0]:
# img = process('images/all_souls_000000.jpg')
img = process('sample.jpeg')

In [0]:
im = Image.fromarray(img)
display(im)

**Window Size of Mask = 6 * sigma** (Optimum) \\
credits: 
1. http://www.sci.utah.edu/~weiliu/class/aip/p1/
2. http://www.cs.unc.edu/~lazebnik/spring11/lec08_blob.pdf
3. [Tutorial on LoG](https://projectsflix.com/opencv/laplacian-blob-detector-using-python/)


In [0]:
k = 2
sigma = 1.0

In [0]:
def LoG(sigma): 
    n = int(np.ceil(sigma * 6)) # Window Size
    values = np.array(range(-n//2, n//2+1)) 
    l = len(values)
    x = values.reshape(1,l)
    y = values.reshape(l,1)
    filter = (-(2*sigma**2) + (x*x + y*y) ) *  (np.exp(-(y*y/(2.*sigma*sigma))) * np.exp(-(x*x/(2.*sigma*sigma)))) * (1/(2*np.pi*sigma**4)) # Scale Normalized LoG
    return filter

In [0]:
LoG_filters = []
n_filters = 9
for i in range(n_filters):
    y = np.power(k,i) 
    scale = sigma * y            
    filt = LoG(scale)           
    LoG_filters.append(filt)

In [0]:
def patch(img, n_count):
    

In [0]:
contrastThreshold = 0.03
def blob_detection(image):
    log_images = []
    for filt in LoG_filters:
        img = cv2.filter2D(src= image, ddepth=cv2.CV_64F, kernel= filt)             # Applied LoG
        img = np.pad(img,((1,1),(1,1)),'constant')      # Padding for 3x3 selection 
        img = np.square(img)                            # Squared response
        log_images.append(img)
    log_images = np.array(log_images)
    
    coordinates = [] 
    height, width = np.shape(image)
    for i in range(1,height):
        for j in range(1,width):
            sliced_imgs = []
            for a in log_images:
                sliced_imgs.append(a[i-1:i+2,j-1:j+2])
            sliced_img = np.array(sliced_image)   # 9*3*3 local slice
            result = np.amax(sliced_img)                 # Finding maximum
            if (result >=contrastThreshold):                        # Contrast Threshold 
                loc = np.where(sliced_img == result)
                z,x,y = loc[0][0], loc[1][0], loc[2][0]
                coordinates.append((i+x-1, j+y-1, (k**z)*sigma))    
    return coordinates

In [0]:
coord = blob_detection(img)

In [0]:
print(coord) # radius = z-axis(coord)*1.414